In [52]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential
from IPython.display import display, Markdown, Latex
import re

In [4]:
def deepseek_chat_completions(prompt, temperature=None, max_tokens=None):
    try:
        endpoint = os.getenv("AZURE_DEEPSEEK_ENDPOINT")
        key = os.getenv("AZURE_DEEPSEEK_KEY")
    except KeyError:
        print("Missing environment variable 'AZURE_DEEPSEEK_ENDPOINT' or 'AZURE_DEEPSEEK_KEY'")
        print("Set them before running this sample.")
        exit()
    client = ChatCompletionsClient(endpoint=endpoint, 
                                   credential=AzureKeyCredential(key), 
                                   model="DeepSeek-V3",
                                   temperature=temperature,
                                   max_tokens=max_tokens)
    

    response = client.complete(
        messages=[
            SystemMessage("You are a helpful assistant. Please add to the response tag <summary></summary> and provide there short summary of the conversational context that could be used to continue the discsussion."),
            UserMessage(prompt),
        ]
    )

    print(response.choices[0].message.content)
    return response

In [5]:
response = deepseek_chat_completions("Tell me please what would be 2*2")

The result of 2 multiplied by 2 is 4.

<summary>Calculation: User asked for the result of 2 multiplied by 2, which is 4. This could be followed by further mathematical questions or explanations of multiplication.</summary>


In [6]:
display(Latex(response.choices[0].message.content))

<IPython.core.display.Latex object>

In [7]:
def extract_latex(text):
    patterns = [
        r'\$\$(.+?)\$\$',  # Block LaTeX with $$...$$
        r'\\\[(.+?)\\\]',  # Block LaTeX with \[...\]
        r'\\\((.+?)\\\)',  # Inline LaTeX with \(...\)
        r'\$(.+?)\$',      # Inline LaTeX with $...$
    ]
    
    latex_matches = []
    for pattern in patterns:
        matches = re.findall(pattern, text, re.DOTALL)  # Use re.DOTALL to match multi-line LaTeX
        latex_matches.extend(matches)
    
    return latex_matches

In [8]:
items = extract_latex(response.choices[0].message.content)

In [9]:
for item in items:
    display(Latex("\[" + item + "\]"))

In [10]:
def split_text_by_latex(text):
    pattern = r'(\$\$(.+?)\$\$|\\\[(.+?)\\\]|\\\((.+?)\\\)|\$(.+?)\$)'
    matches = re.finditer(pattern, text, re.DOTALL)
    
    result = []
    last_end = 0
    
    for match in matches:
        start, end = match.span()
        
        if start > last_end:
            result.append(('text', text[last_end:start]))
        
        result.append(('latex', match.group(0)))
        
        last_end = end
    
    if last_end < len(text):
        result.append(('text', text[last_end:]))
    
    return result

In [19]:
def process_model_response(response):
    items = split_text_by_latex(response)
    for item in items:
        if item[0] == 'latex':
            display(Latex(item[1]))
        elif item[0] == 'text':
            display(Markdown(item[1]))
        else:
            raise Exception("Unsupported type: " + item[0])

In [21]:
process_model_response(response.choices[0].message.content)

The result of 2 multiplied by 2 is 4.

<summary>Calculation: User asked for the result of 2 multiplied by 2, which is 4. This could be followed by further mathematical questions or explanations of multiplication.</summary>

In [22]:
response

{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'The result of 2 multiplied by 2 is 4.\n\n<summary>Calculation: User asked for the result of 2 multiplied by 2, which is 4. This could be followed by further mathematical questions or explanations of multiplication.</summary>', 'reasoning_content': None, 'role': 'assistant', 'tool_calls': None}}], 'created': 1741556718, 'id': 'dc7be1bed693426abdc3dbfeca60385a', 'model': 'DeepSeek-V3', 'object': 'chat.completion', 'usage': {'completion_tokens': 52, 'prompt_tokens': 49, 'prompt_tokens_details': None, 'total_tokens': 101}}

In [18]:
def deepseek_chat_completions_v2(prompt, temperature=None, max_tokens=None):
    try:
        endpoint = os.getenv("AZURE_DEEPSEEK_ENDPOINT")
        key = os.getenv("AZURE_DEEPSEEK_KEY")
    except KeyError:
        print("Missing environment variable 'AZURE_DEEPSEEK_ENDPOINT' or 'AZURE_DEEPSEEK_KEY'")
        print("Set them before running this sample.")
        exit()
    client = ChatCompletionsClient(endpoint=endpoint, 
                                   credential=AzureKeyCredential(key), 
                                   model="DeepSeek-V3",
                                   temperature=temperature,
                                   max_tokens=max_tokens)
    

    response = client.complete(
        messages=[
            SystemMessage("You are a helpful assistant. Please add to the response tag <summary></summary> and provide there short summary of the conversational context that could be used to continue the discsussion."),
            UserMessage(prompt),
        ],
        stream=True
    )
    
    message_content_items = []
    for chunk in response:
        # Extract the delta content from the chunk
        if "choices" in chunk and chunk["choices"]:
            delta = chunk["choices"][0]["delta"]
            if "content" in delta:
                message_content_items.append(delta["content"])
                print(delta["content"], end="", flush=True)
                
    message_content = "".join(message_content_items)          
    
    print(response)
    
    return response, message_content

In [23]:
task = "There is a table with 12 people. Some of them liers and some of them knights.\
        Liers can only lie, knights can only speak truth. Each of them told that he has\
        a lier on the right side. How many liers and how many knights?"
response, content = deepseek_chat_completions_v2(task)

Let's analyze the problem systematically:

1. **Understanding the Setup:**
   - There are **12 people** sitting around a table.
   - Each person is either a **knight** (always tells the truth) or a **liar** (always lies).
   - Each person claims, "I have a liar on my right side."

2. **Breaking Down the Statements:**
   - If a person is a **knight**, their statement is **true**. Therefore, the person to their right must indeed be a liar.
   - If a person is a **liar**, their statement is **false**. Therefore, the person to their right cannot be a liar; they must be a knight.

3. **Creating a Sequence:**
   - Let's denote knights as **K** and liars as **L**.
   - If we start with a knight (K), the next person to the right must be a liar (L).
   - Following this pattern:
     - K → L → K → L → K → L → K → L → K → L → K → L

4. **Verifying the Pattern:**
   - Each knight correctly identifies a liar to their right.
   - Each liar falsely claims the person to their right is a liar when it i

In [24]:
process_model_response(content)

Let's analyze the problem systematically:

1. **Understanding the Setup:**
   - There are **12 people** sitting around a table.
   - Each person is either a **knight** (always tells the truth) or a **liar** (always lies).
   - Each person claims, "I have a liar on my right side."

2. **Breaking Down the Statements:**
   - If a person is a **knight**, their statement is **true**. Therefore, the person to their right must indeed be a liar.
   - If a person is a **liar**, their statement is **false**. Therefore, the person to their right cannot be a liar; they must be a knight.

3. **Creating a Sequence:**
   - Let's denote knights as **K** and liars as **L**.
   - If we start with a knight (K), the next person to the right must be a liar (L).
   - Following this pattern:
     - K → L → K → L → K → L → K → L → K → L → K → L

4. **Verifying the Pattern:**
   - Each knight correctly identifies a liar to their right.
   - Each liar falsely claims the person to their right is a liar when it is actually a knight.
   - This creates an alternating sequence that satisfies all statements.

5. **Counting Knights and Liars:**
   - In a sequence of 12 people with an alternating pattern (K, L, K, L, ...), there are:
     - **6 knights**
     - **6 liars**

6. **Conclusion:**
   - The only consistent arrangement that satisfies all given statements is an alternating sequence of knights and liars.
   - Therefore, there are **6 knights** and **6 liars** at the table.

**Final Answer:** There are **6 knights** and **6 liars** at the table.

<summary>The problem involves determining the number of knights and liars seated around a table, given that each person claims to have a liar on their right. By analyzing the truthfulness of their statements and establishing an alternating pattern, it is concluded that there are 6 knights and 6 liars.</summary>

In [17]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
enc.encode("Hello world!")

[13225, 2375, 0]

In [ ]:
file_path = './data/ai_book_sample.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    book_text = file.read()

In [47]:
book_tokens = enc.encode(book_text)

In [48]:
len(book_tokens)

169887

In [43]:
book_prompt = "Let's read this book, please do a short\
                summary, then summary for chapters and finally\
                let's try to distill if there are some important & new / novel ideas and knowledge\
                (business or technological). Let's skip all buzzwords or other things\
                that look just a copy paste over the internet."

In [64]:
def deepseek_chat_completions_v3_document(prompt, document, temperature=None, max_tokens=None, chunk_size=1024):
    try:
        endpoint = os.getenv("AZURE_DEEPSEEK_ENDPOINT")
        key = os.getenv("AZURE_DEEPSEEK_KEY")
    except KeyError:
        print("Missing environment variable 'AZURE_DEEPSEEK_ENDPOINT' or 'AZURE_DEEPSEEK_KEY'")
        print("Set them before running this sample.")
        exit()
    client = ChatCompletionsClient(endpoint=endpoint, 
                                   credential=AzureKeyCredential(key), 
                                   model="DeepSeek-V3",
                                   temperature=temperature,
                                   max_tokens=max_tokens)
    

    chunks = [document[i:i + chunk_size] for i in range(0, len(document), chunk_size)]
    messages = []
    responses = []
    message_content_items = []
    messages.append(
        SystemMessage(
                """
                You are a helpful assistant that can process queries and large, multi-chunk documents. 
                Please add a summary to your response, enclosed in the tags <summary></summary>, 
                providing a short summary of the conversational context that can be used to continue the discussion.
                I will provide the document in chunks, starting with the tag <document>. 
                Each subsequent chunk will be prefixed with the tag <document_chunk>. 
                Do not generate any response until I send a final message with the tag </document>. 
                Once you receive that message, process the entire document and generate a comprehensive response.
                """
        )
    )
    
    for i, chunk in enumerate(chunks):
        print(f"Chunk: {i}")
        if i == 0:
            messages.append(UserMessage(f"<document><document_chunk>{chunk}"))
        elif i == len(chunks) - 1:
            messages.append(UserMessage(f"<document_chunk>{chunk}</document>"))
        else:
            messages.append(UserMessage(f"<document_chunk>{chunk}"))
        
    response = client.complete(messages=messages, stream=True)
    responses.append(response)
    for chunk in response:
        if "choices" in chunk and chunk["choices"]:
            delta = chunk["choices"][0]["delta"]
            if "content" in delta:
                message_content_items.append(delta["content"])
                messages.append(AssistantMessage(delta["content"]))
                print(delta["content"], end="", flush=True)
        
    message_content = "".join(message_content_items)          
    
    print(responses)
    
    return responses, message_content

In [65]:
response, content = deepseek_chat_completions_v3_document(book_prompt, book_text)
process_model_response(content)

Chunk: 0
Chunk: 1
Chunk: 2
Chunk: 3
Chunk: 4
Chunk: 5
Chunk: 6
Chunk: 7
Chunk: 8
Chunk: 9
Chunk: 10
Chunk: 11
Chunk: 12
Chunk: 13
Chunk: 14
Chunk: 15
Chunk: 16
Chunk: 17
Chunk: 18
Chunk: 19
Chunk: 20
Chunk: 21
Chunk: 22
Chunk: 23
Chunk: 24
Chunk: 25
Chunk: 26
Chunk: 27
Chunk: 28
Chunk: 29
Chunk: 30
Chunk: 31
Chunk: 32
Chunk: 33
Chunk: 34
Chunk: 35
Chunk: 36
Chunk: 37
Chunk: 38
Chunk: 39
Chunk: 40
Chunk: 41
Chunk: 42
Chunk: 43
Chunk: 44
Chunk: 45
Chunk: 46
Chunk: 47
Chunk: 48
Chunk: 49
Chunk: 50
Chunk: 51
Chunk: 52
Chunk: 53
Chunk: 54
Chunk: 55
Chunk: 56
Chunk: 57
Chunk: 58
Chunk: 59
Chunk: 60
Chunk: 61
Chunk: 62
Chunk: 63
Chunk: 64
Chunk: 65
Chunk: 66
Chunk: 67
Chunk: 68
Chunk: 69
Chunk: 70
Chunk: 71
Chunk: 72
Chunk: 73
Chunk: 74
Chunk: 75
Chunk: 76
Chunk: 77
Chunk: 78
Chunk: 79
Chunk: 80
Chunk: 81
Chunk: 82
Chunk: 83
Chunk: 84
Chunk: 85
Chunk: 86
Chunk: 87
Chunk: 88
Chunk: 89
Chunk: 90
Chunk: 91
Chunk: 92
Chunk: 93
Chunk: 94
Chunk: 95
Chunk: 96
Chunk: 97
Chunk: 98
Chunk: 99
Chunk: 100

HttpResponseError: (tokens_limit_reached) Request body too large for deepseek-v3 model. Max size: 8000 tokens.
Code: tokens_limit_reached
Message: Request body too large for deepseek-v3 model. Max size: 8000 tokens.

In [77]:
import os

# Function to estimate token count (assuming OpenAI-style tokenizer)
def estimate_token_count(text):
    return len(text.split())  # Rough approximation (better: use tiktoken for accuracy)

def deepseek_chat_completions_v4_document(prompt, document, temperature=None, max_tokens=None, chunk_size=800):
    try:
        endpoint = os.getenv("AZURE_DEEPSEEK_ENDPOINT")
        key = os.getenv("AZURE_DEEPSEEK_KEY")
    except KeyError:
        print("Missing environment variable 'AZURE_DEEPSEEK_ENDPOINT' or 'AZURE_DEEPSEEK_KEY'")
        exit()
    
    client = ChatCompletionsClient(
        endpoint=endpoint, 
        credential=AzureKeyCredential(key), 
        model="DeepSeek-V3",
        temperature=temperature,
        max_tokens=max_tokens
    )

    chunks = [document[i:i + chunk_size] for i in range(0, len(document), chunk_size)]
    messages = []
    responses = []

    # **System Instruction Message (Base Prompt)**
    system_message = SystemMessage(
        """
        You are a helpful assistant that can process queries and large, multi-chunk documents. 
        Please add a summary to your response, enclosed in the tags <summary></summary>, 
        providing a short summary of the conversational context that can be used to continue the discussion.
        I will provide the document in chunks, starting with the tag <document>. 
        Each subsequent chunk will be prefixed with the tag <document_chunk>. 
        Do not generate any response until I send a final message with the tag </document>. 
        Once you receive that message, process the entire document and generate a comprehensive response.
        """
    )

    messages.append(system_message)

    # **Process in Batches to Stay Under 8000 Tokens**
    accumulated_text = ""
    total_tokens = estimate_token_count(system_message.content)

    for i, chunk in enumerate(chunks):
        print(f"chunk: {i}")
        
        chunk_tokens = estimate_token_count(chunk)

        # If adding this chunk exceeds 2000 tokens, send the accumulated batch
        if total_tokens + chunk_tokens > 2000:
            messages.append(UserMessage("</document>"))  # Mark end of batch
            response = client.complete(messages=messages, stream=True)
            
            # **Stream the response live**
            streamed_response = []
            for chunk in response:
                if "choices" in chunk and chunk["choices"]:
                    delta = chunk["choices"][0]["delta"]
                    if "content" in delta:
                        streamed_response.append(delta["content"])
                        print(delta["content"], end="", flush=True)  # Stream output in real-time

            responses.append("".join(streamed_response))  # Store streamed output

            # Reset messages with only system message for the next batch
            messages = [system_message]
            total_tokens = estimate_token_count(system_message.content)
        
        # Add chunk and update token count
        if i == 0:
            messages.append(UserMessage(f"<document>\n<document_chunk>{chunk}"))
        else:
            messages.append(UserMessage(f"<document_chunk>{chunk}"))

        total_tokens += chunk_tokens

    # **Final batch processing with stream=True**
    messages.append(UserMessage("</document>"))
    response = client.complete(messages=messages, stream=True)

    streamed_response = []
    for chunk in response:
        if "choices" in chunk and chunk["choices"]:
            delta = chunk["choices"][0]["delta"]
            if "content" in delta:
                streamed_response.append(delta["content"])
                print(delta["content"], end="", flush=True)  # Stream output in real-time

    responses.append("".join(streamed_response))  # Store streamed output

    return "\n".join(responses)  # Combine results from multiple batches

In [ ]:
response, content = deepseek_chat_completions_v4_document(book_prompt, book_text)
process_model_response(content)

chunk: 0
chunk: 1
chunk: 2
chunk: 3
chunk: 4
chunk: 5
chunk: 6
chunk: 7
chunk: 8
chunk: 9
chunk: 10
chunk: 11
chunk: 12
chunk: 13
chunk: 14
chunk: 15
"The Coming Wave" by Mustafa Suleyman is a highly anticipated book that delves into the transformative and potentially perilous impact of advanced technologies, particularly artificial intelligence (AI) and synthetic biology. The book has garnered widespread acclaim from thought leaders, technologists, and policymakers for its insightful analysis and urgent call to action. 

Suleyman, a prominent figure in the tech industry and co-founder of AI companies like DeepMind and Inflection, explores the dual-edged nature of these technologies. While they promise unprecedented benefits—such as advancements in medicine, productivity, and innovation—they also pose existential risks, including geopolitical upheaval, loss of control, and societal disruption. The book emphasizes the "containment challenge," arguing that humanity must develop robust go

The book aims to explore these issues in detail, examining the historical context of technological waves, the specific risks posed by AI and synthetic biology, and the inherent difficulties in containing such general-purpose technologies. The author urges readers to confront these realities head-on, despite the discomfort they may cause, and to consider the broader implications for humanity.

<summary>The document highlights the urgent need to address the risks posed by rapidly advancing technologies like AI and synthetic biology, emphasizing the importance of societal and political responses to ensure containment and safety. The author reflects on the tendency to avoid confronting these challenges (pessimism aversion) and calls for a more proactive approach to managing technological innovation.</summary>chunk: 60
chunk: 61
chunk: 62
chunk: 63
chunk: 64
chunk: 65
chunk: 66
chunk: 67
chunk: 68
chunk: 69
chunk: 70
chunk: 71
chunk: 72
chunk: 73
chunk: 74
The document discusses the evoluti

The document argues that technological waves are nearly impossible to stop due to their inherent appeal and the demand they generate. It highlights how older technologies are often replaced by newer, more efficient forms, such as the transition from candles to electric lights or from typewriters to computers. However, it also examines a notable exception: nuclear weapons. While nuclear technology has the potential for catastrophic destruction, its proliferation has been relatively contained due to the complexity and cost of production, as well as international treaties like the Non-Proliferation Treaty of 1968.

<summary>The document discusses historical resistance to technological advancements, showing how societies have often failed to contain new technologies due to their widespread appeal and demand. It uses examples like the printing press, industrial machinery, and nuclear weapons to illustrate that while containment efforts have been attempted, they are rarely successful. The ex

1. **AI Applications**: AI is being utilized across various sectors including energy (modeling fusion reactions), transportation (optimizing routes and driving vehicles), and infrastructure (designing sustainable materials). It also plays a role in managing resources like electrical grids and water systems, as well as in retail, healthcare, fraud detection, and simulating climate change impacts.

2. **Large Language Models (LLMs)**: The rise of LLMs, particularly since the introduction of ChatGPT by OpenAI in November 2022, has marked a significant leap in AI capabilities. These models can process and generate natural language efficiently, perform tasks like writing essays, creating business plans, and even coding.

3. **Technology Behind LLMs**: The core technology involves transformers, which use attention mechanisms to process language data sequentially. These models learn from vast amounts of text data, creating an "attention map" to understand and predict language patterns.

4. **

- **Transformative Potential**: AI is not just a tool but a meta-technology that will generate new systems and platforms, fundamentally transforming industries and society.

### Synthetic Biology and Genetic Engineering
- **DNA as Information**: DNA is recognized as an information storage system, enabling interventions to alter genetic encoding and direct biological processes.
- **CRISPR Revolution**: Modern genetic engineering, built on foundational discoveries like the structure of DNA, has become more accessible and cost-effective. The Human Genome Project was a landmark achievement that made the human genome legible, paving the way for innovations in medicine, food, materials, and manufacturing.
- **Cost Reduction**: The cost of genome sequencing has dropped dramatically, from $1 billion in 2003 to under $1,000 by 2022, enabling widespread applications.

### Conclusion
The document emphasizes that both AI and synthetic biology represent transformative technologies with profound imp

The narrative concludes with a dramatic example of a police robot being used to neutralize a sniper in Dallas, underscoring the increasing presence of robots in sensitive and extreme situations. The document emphasizes that while robots may not resemble the humanoid figures of science fiction, they are becoming ubiquitous and are poised to play a significant role in society, bridging the gap between AI and the physical world.

<summary>The document discusses the integration of robotics and AI across various industries, from agriculture to law enforcement, highlighting their transformative potential. It explores the history of agricultural technology, the challenges of developing adaptable robots, and the future of robotics in society, including the concept of robot swarms. The narrative concludes with an example of a police robot being used in a critical situation, illustrating the growing presence of robots in daily life.</summary>chunk: 264
chunk: 265
chunk: 266
chunk: 267
chunk: 268

The document discusses the rapid evolution of artificial intelligence (AI) and its potential to surpass human understanding and control. It highlights the challenges posed by autonomous systems, which are often "black boxes" with decision-making processes that are difficult to explain or predict. The discussion extends to the potential for AI to recursively improve itself, leading to an "intelligence explosion" that could render it uncontrollable. This raises concerns about humanity's ability to contain such advanced technologies, especially if they become superintelligent.

The document also touches on the societal and geopolitical implications of AI advancements, using the example of AlphaGo's victory over human Go champions as a case study. This event was not just a technological milestone but also a moment of national pride and competition, particularly in Asia. The narrative underscores how AI development is driven by powerful incentives, including great power competition, curiosi

The document discusses the transformative potential of emerging technologies, particularly artificial intelligence (AI), and their implications for the global economy, agriculture, climate change, and other critical challenges. It highlights how AI and other innovations could significantly boost economic growth, improve agricultural productivity, and address environmental issues. The text also emphasizes the necessity of technological advancements to meet global challenges such as food security, climate change, and healthcare demands, while acknowledging the complexities and moral imperatives involved in developing and deploying these technologies.

<summary>The document explores the profound impact of emerging technologies like AI on global economic growth, agriculture, climate change, and healthcare. It underscores the necessity and potential of these technologies to address critical global challenges while acknowledging the complexities and ethical considerations involved in their d